In [1]:
import pandas as pd

# نام فایل خودت را جایگزین کن
df = pd.read_csv("Data\Posts_ikea_16_Oct_2025_14_Nov_2025_1b0e.csv")

print(df.head())
print(df.columns)


   Name                         Profile  \
0  ikea  https://www.instagram.com/ikea   
1  ikea  https://www.instagram.com/ikea   
2  ikea  https://www.instagram.com/ikea   
3  ikea  https://www.instagram.com/ikea   
4  ikea  https://www.instagram.com/ikea   

                                     Permalink  \
0     https://www.instagram.com/p/DQWZuZGkz3P/   
1     https://www.instagram.com/p/DQuCeK8ieO3/   
2     https://www.instagram.com/p/DQq9yeUjQxz/   
3  https://www.instagram.com/reel/DQL8rHXgE1B/   
4  https://www.instagram.com/reel/DP3dZNHABfm/   

                                             Caption            Type  \
0  A cosy corner for when you need a calm moment ...           IMAGE   
1  Smart products now simpler than ever ✨⁠⁠We’re ...  CAROUSEL_ALBUM   
2  Making waste sorting a little easier ✨ Say hej...  CAROUSEL_ALBUM   
3  For everyone at the table kids grown-ups and e...            reel   
4  It’s not about perfect plates or fancy recipes...            reel   

       

In [2]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# اگر دوست داری ستون‌های روز و ساعت هم داشته باشی:
df['DayOfWeek'] = df['Date'].dt.day_name()
df['Hour'] = df['Date'].dt.hour


In [3]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"   # 1 تا 5 ستاره
)


C:\Users\Nastaran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Device set to use cpu


In [4]:
texts = df['Caption'].fillna("").astype(str).tolist()
batch_size = 32

labels = []
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    preds = sentiment_model(batch)
    labels.extend(preds)

df['bert_sentiment_label'] = [x['label'] for x in labels]
df['bert_sentiment_score'] = [x['score'] for x in labels]

# نگاشت 1 تا 5 ستاره به منفی / خنثی / مثبت
def stars_to_polarity(label):
    n = int(label.split()[0])  # "1 star" → 1
    if n <= 2:
        return "negative"
    elif n == 3:
        return "neutral"
    else:
        return "positive"

df['bert_sentiment_polarity'] = df['bert_sentiment_label'].apply(stars_to_polarity)

print(df[['Caption', 'bert_sentiment_label', 'bert_sentiment_polarity']].head())


                                             Caption bert_sentiment_label  \
0  A cosy corner for when you need a calm moment ...              4 stars   
1  Smart products now simpler than ever ✨⁠⁠We’re ...              5 stars   
2  Making waste sorting a little easier ✨ Say hej...              5 stars   
3  For everyone at the table kids grown-ups and e...              5 stars   
4  It’s not about perfect plates or fancy recipes...              3 stars   

  bert_sentiment_polarity  
0                positive  
1                positive  
2                positive  
3                positive  
4                 neutral  


In [9]:
print("Number of documents:", len(docs))


Number of documents: 9


In [12]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

# فقط ردیف‌هایی که کپشن دارند
df_topic = df[df['Caption'].notna()].copy()
docs = df_topic['Caption'].astype(str).tolist()

n_docs = len(docs)
print("Number of documents:", n_docs)

if n_docs < 3:
    print("📉 تعداد پست‌ها خیلی کمه، تاپیک مدلینگ معنی‌دار نمیشه.")
else:
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

    # برای دیتاست کوچک: n_neighbors باید < تعداد سندها باشه
    n_neighbors = max(2, min(5, n_docs - 1))   # برای ۹ تا پست میشه 5

    umap_model = UMAP(
        n_neighbors=n_neighbors,
        n_components=2,
        metric="cosine",
        random_state=42
    )

    # HDBSCAN پیش‌فرض min_cluster_size=10 داره که از ۹ بیشتره!
    # برای همین خودمون کوچیکش می‌کنیم:
    hdbscan_model = HDBSCAN(
        min_cluster_size=2,
        metric="euclidean",
        cluster_selection_method="eom",
        prediction_data=True
    )

    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        language="english",
        verbose=True
    )

    topics, probs = topic_model.fit_transform(docs)
    df_topic['topic'] = topics

    print(df_topic[['Caption', 'topic']])


Number of documents: 9


2025-11-14 13:44:30,599 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-14 13:44:31,859 - BERTopic - Embedding - Completed ✓
2025-11-14 13:44:31,863 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-14 13:44:31,993 - BERTopic - Dimensionality - Completed ✓
2025-11-14 13:44:31,999 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-14 13:44:32,026 - BERTopic - Cluster - Completed ✓
2025-11-14 13:44:32,061 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-14 13:44:32,160 - BERTopic - Representation - Completed ✓


                                             Caption  topic
0  A cosy corner for when you need a calm moment ...     -1
1  Smart products now simpler than ever ✨⁠⁠We’re ...     -1
2  Making waste sorting a little easier ✨ Say hej...     -1
3  For everyone at the table kids grown-ups and e...     -1
4  It’s not about perfect plates or fancy recipes...     -1
5              Making lifelong memories (and mess) 💛     -1
6  Its IKEA Foundation Week and were highlighting...     -1
7  Passion makes every kitchen limitless - flavou...     -1
8            Everyday comfort made simple 💚#BROKGLIM     -1


In [16]:
topics, probs = topic_model.fit_transform(docs)
df_topic['topic'] = topics
print(df_topic[['Caption', 'topic']])


2025-11-14 13:46:35,157 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-14 13:46:37,147 - BERTopic - Embedding - Completed ✓
2025-11-14 13:46:37,151 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-14 13:46:37,418 - BERTopic - Dimensionality - Completed ✓
2025-11-14 13:46:37,422 - BERTopic - Cluster - Start clustering the reduced embeddings


ValueError: k must be less than or equal to the number of training points

In [15]:
topic_info = topic_model.get_topic_info()
print(topic_info)


ValueError: This BERTopic instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def clean_keywords(words):
    cleaned = []
    for w in words:
        w_clean = re.sub(r'[^a-zA-Z]', '', w).lower()
        if len(w_clean) < 3:
            continue
        if w_clean in STOPWORDS:
            continue
        cleaned.append(w_clean)
    return cleaned

def select_main_words(words):
    tagged = pos_tag(words)
    filtered = [w for w, tag in tagged if tag.startswith('NN') or tag.startswith('VB')]
    return filtered if filtered else words

def make_better_title(words):
    if not words:
        return "Unknown Topic"
    words = clean_keywords(words)
    words = select_main_words(words)
    if not words:
        return "Unnamed Topic"
    if len(words) == 1:
        return words[0].capitalize()
    if len(words) == 2:
        return f"{words[0].capitalize()} & {words[1].capitalize()}"
    return f"{words[0].capitalize()} {words[1].capitalize()}"

topic_titles = {}
for tid in topic_info['Topic'].tolist():
    if tid == -1:
        continue
    words_raw = topic_model.get_topic(tid)
    if words_raw is None:
        continue
    top_words = [w[0] for w in words_raw[:10]]
    topic_titles[tid] = make_better_title(top_words)

print("Sample titles:")
for k, v in list(topic_titles.items())[:10]:
    print(k, ":", v)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nastaran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nastaran\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nastaran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'topic_info' is not defined

In [ ]:
# مطمئن شو df_topic همه ستون‌های عددی را دارد
df_topic = df[df['topic'].notna()].copy()

# میانگین‌ها
likes_by_topic = df_topic.groupby('topic')['Likes'].mean().round(2)
comments_by_topic = df_topic.groupby('topic')['Comments'].mean().round(2)
engrate_by_topic = df_topic.groupby('topic')['Eng. Rate by Followers'].mean().round(4)

# احساس غالب هر topic
sent_df = (
    df_topic
    .groupby(['topic', 'bert_sentiment_polarity'])
    .size()
    .reset_index(name='count')
)

dominant_sentiment = {}
for tid in sent_df['topic'].unique():
    temp = sent_df[sent_df['topic'] == tid]
    dom = temp.sort_values('count', ascending=False).iloc[0]['bert_sentiment_polarity']
    dominant_sentiment[tid] = dom

# ساخت جدول
report_rows = []
for tid in sorted(df_topic['topic'].unique()):
    if tid == -1:
        continue
    report_rows.append({
        "Topic ID": tid,
        "Title": topic_titles.get(tid, ""),
        "Keywords": ", ".join([w[0] for w in (topic_model.get_topic(tid) or [])[:5]]),
        "Avg Likes": likes_by_topic.get(tid, 0),
        "Avg Comments": comments_by_topic.get(tid, 0),
        "Avg Eng. Rate": engrate_by_topic.get(tid, 0),
        "Dominant Sentiment": dominant_sentiment.get(tid, "")
    })

topic_report = pd.DataFrame(report_rows).sort_values("Avg Likes", ascending=False)
print(topic_report)


In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

import re

def caption_to_hashtags(text, top_n=5):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 2),
        stop_words='english',
        top_n=top_n
    )
    tags = []
    for kw, score in keywords:
        kw_clean = re.sub(r'[^a-zA-Z0-9\s]', '', kw)
        if not kw_clean:
            continue
        tag = "#" + kw_clean.replace(" ", "").lower()
        tags.append(tag)
    # یکتا
    tags = list(dict.fromkeys(tags))
    return " ".join(tags)

df['suggested_hashtags'] = df['Caption'].apply(caption_to_hashtags)

print(df[['Caption', 'suggested_hashtags']].head())


In [ ]:
print(df.groupby('Type')['Engagement'].mean().sort_values(ascending=False))


In [ ]:
print(df.groupby('Content Pillars')['Eng. Rate by Followers'].mean().sort_values(ascending=False))


In [ ]:
print(df.groupby('bert_sentiment_polarity')['Likes'].mean())
